# Taxonomic analysis 

Preambule to Multiple Sequence alignment and Phylogenetic tree building.

### Import necessary modules 

In [1]:
from Bio import (
    SeqIO as seqio, 
    SearchIO as searchio, 
    Entrez as entrez
)

from Bio.Seq import Seq as seq

import toml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial 

from annotathon.utils.customobjs import (
    Path as path,
    objdict as odict
)

from annotathon.parsing.blast import parse_alignment_descriptions as parse_ncbi
from annotathon.annotation.helper_functions import *

### Configuration to access NCBI's servers :

In [2]:
# Load configuration to access NCBI :
with open("../creds/entrezpy.toml", "r") as f:
    ncbi = toml.load(f, _dict=odict)

In [3]:
# set credentials : 
entrez.api_key = ncbi.credentials.apikey
entrez.email = ncbi.credentials.email
entrez.tool = ncbi.credentials.tool

In [4]:
# set plotting params :
%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (15, 8)

In [5]:
with open("../config/locations.toml", "r") as f:
    _config = toml.load(f, _dict=odict)
_config

{'locations': {'default': '../data/homologues-par-blast/default-nr',
  'cinqk': '../data/homologues-par-blast/5k-nr',
  'landmark': '../data/homologues-par-blast/landmark-noexclude',
  'sp': '../data/homologues-par-blast/swissprot',
  'anthony': '../data/anthony/'}}

In [6]:
locations = odict({ 
    key: path(value) for key, value in _config.locations.items()
})
locations

{'default': PosixPath('../data/homologues-par-blast/default-nr'),
 'cinqk': PosixPath('../data/homologues-par-blast/5k-nr'),
 'landmark': PosixPath('../data/homologues-par-blast/landmark-noexclude'),
 'sp': PosixPath('../data/homologues-par-blast/swissprot'),
 'anthony': PosixPath('../data/anthony')}

In [7]:
blast = odict({
    "locations": locations,
    "data": odict({})
})
blast

{'locations': {'default': PosixPath('../data/homologues-par-blast/default-nr'),
  'cinqk': PosixPath('../data/homologues-par-blast/5k-nr'),
  'landmark': PosixPath('../data/homologues-par-blast/landmark-noexclude'),
  'sp': PosixPath('../data/homologues-par-blast/swissprot'),
  'anthony': PosixPath('../data/anthony')},
 'data': {}}

In [8]:
description_glob = "*Alignment-Descriptions*"

### Load blast alignment descriptions

In [9]:
blast.data.update(odict({
    "default": parse_ncbi(blast.locations.default.lglob(description_glob)[0]),
    "cinqk": parse_ncbi(blast.locations.cinqk.lglob(description_glob)[0]),
    "landmark": parse_ncbi(blast.locations.landmark.lglob(description_glob)[0]),
    "sp": parse_ncbi(blast.locations.sp.lglob(description_glob)[0]),
    "taxo": parse_ncbi(blast.locations.anthony.lglob(description_glob)[0]),
    "hypo": parse_ncbi(blast.locations.anthony.lglob("*.csv")[1])
}))

This is probably unnecessary given that we now have all the information from the genbank !

In [10]:
#blast.data.taxo.loc[:, "Description"] = add_function(blast.data.taxo.Description)
blast.data.taxo.loc[:, "function"] = add_function(blast.data.taxo.Description)
blast.data.taxo.loc[:, "species"] = add_species(blast.data.taxo.Description)

In [11]:
#blast.data.cinqk.loc[:, "Description"] = add_function(blast.data.cinqk.Description)
blast.data.cinqk.loc[:, "function"] = add_function(blast.data.cinqk.Description)
blast.data.cinqk.loc[:, "species"] = add_species(blast.data.cinqk.Description)

In [12]:
tests = pd.read_csv("efetch-tests.csv")

In [13]:
with entrez.efetch(db="protein", id=tests.Accession.to_list(), rettype="gb", retmode="text") as finallyyeah:
    y = [ entry for entry in seqio.parse(finallyyeah, format="gb") ]

In [22]:
y[0].annotations

{'topology': 'linear',
 'data_file_division': 'ENV',
 'date': '01-SEP-2020',
 'accessions': ['MBC8297997'],
 'sequence_version': 1,
 'db_source': 'accession JACNKM010000857.1',
 'keywords': ['ENV'],
 'source': 'Pelagibacterales bacterium (marine metagenome)',
 'organism': 'Pelagibacterales bacterium',
 'taxonomy': ['Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'Pelagibacterales',
  'unclassified Pelagibacterales'],
 'references': [Reference(title='Bridging the membrane lipid divide: bacteria of the FCB group superphylum have the potential to synthesize archaeal ether lipids', ...),
  Reference(title='Direct Submission', ...)],
 'comment': 'The annotation was added by the NCBI Prokaryotic Genome Annotation\nPipeline (PGAP). Information about PGAP can be found here:\nhttps://www.ncbi.nlm.nih.gov/genome/annotation_prok/',
 'structured_comment': OrderedDict([('Genome-Assembly-Data',
               OrderedDict([('Assembly Method', 'metaSPAdes v. v3.8.0'),
                      

In [23]:
for i in y:
    print(len(i.annotations["accessions"]))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [14]:
for i in y:
    print(i.annotations["taxonomy"])

['Bacteria', 'Proteobacteria', 'Alphaproteobacteria', 'Pelagibacterales', 'unclassified Pelagibacterales']
[]
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Firmicutes', 'Erysipelotrichia', 'Erysipelotrichales', 'Erysipelotrichaceae', 'Turicibacter']
['Bacteria', 'Proteobacteria', 'Gammaproteobacteria']


In [78]:
# SAVE AS GENEBANK ! 
# FASTA LOSES LOTS OF INFORMATION !
blast.data.cinqk.head()

,Description,Common Name,Max Score,Total Score,Query Cover,E value,Per. ident,Acc. Len,Accession,function,species
0,hypothetical protein [Gammaproteobacteria bact...,Gammaproteobacteria bacterium,672,672,100.0,0.000000e+00,95.16,687,MAO06092.1,hypothetical protein,Gammaproteobacteria bacterium
1,hypothetical protein [Gammaproteobacteria bact...,Gammaproteobacteria bacterium,662,662,100.0,0.000000e+00,94.02,687,MBS55433.1,hypothetical protein,Gammaproteobacteria bacterium
2,hypothetical protein [Gammaproteobacteria bact...,Gammaproteobacteria bacterium,303,303,100.0,5.000000e-95,42.90,553,MBK47166.1,hypothetical protein,Gammaproteobacteria bacterium
3,ATP-dependent DNA helicase RecG [bacterium TME...,bacterium TMED221,306,306,100.0,2.000000e-94,44.60,690,RPH01348.1,ATP-dependent DNA helicase RecG,bacterium TMED221
4,hypothetical protein [Gammaproteobacteria bact...,Gammaproteobacteria bacterium,300,300,99.0,2.000000e-92,43.59,692,MBJ40943.1,hypothetical protein,Gammaproteobacteria bacterium


In [79]:
with entrez.efetch(db="protein", id=blast.data.cinqk.Accession.to_list(), rettype="gb", retmode="text") as in_handle:
    with open("5k-info.gb", "w") as out_handle:
        sequences = seqio.parse(in_handle, format="gb")
        seqio.write(sequences, out_handle, format="gb") 

In [61]:
# some dummy test : 
with entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text") as wow:
    print(wow.read())

LOCUS       EU490707                1302 bp    DNA     linear   PLN 26-JUL-2016
DEFINITION  Selenipedium aequinoctiale maturase K (matK) gene, partial cds;
            chloroplast.
ACCESSION   EU490707
VERSION     EU490707.1
KEYWORDS    .
SOURCE      chloroplast Selenipedium aequinoctiale
  ORGANISM  Selenipedium aequinoctiale
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliopsida; Liliopsida; Asparagales; Orchidaceae;
            Cypripedioideae; Selenipedium.
REFERENCE   1  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A., Endara,L.,
            Williams,N.H. and Moore,M.
  TITLE     Phylogenetic utility of ycf1 in orchids: a plastid gene more
            variable than matK
  JOURNAL   Plant Syst. Evol. 277 (1-2), 75-84 (2009)
REFERENCE   2  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A.,
            Endara,C.L., Williams,N.H. and Moore,M.J.
  TIT

In [18]:
help(entrez.efetch)

Help on function efetch in module Bio.Entrez:

efetch(db, **keywords)
    Fetch Entrez results which are returned as a handle.
    
    EFetch retrieves records in the requested format from a list or set of one or
    more UIs or from user's environment.
    
    See the online documentation for an explanation of the parameters:
    http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch
    
    Return a handle to the results.
    
    Raises an IOError exception if there's a network error.
    
    Short example:
    
    >>> from Bio import Entrez
    >>> Entrez.email = "Your.Name.Here@example.org"
    >>> handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
    >>> print(handle.readline().strip())
    LOCUS       AY851612                 892 bp    DNA     linear   PLN 10-APR-2007
    >>> handle.close()
    
    This will automatically use an HTTP POST rather than HTTP GET if there
    are over 200 identifiers as recommended by the NCBI.
    
    